In [11]:
import sys
from pathlib import Path

# project root = parent folder of "notebooks"
ROOT = Path().resolve().parents[0]
# falls dein Notebook wirklich im notebooks-Ordner läuft, ist ROOT jetzt das Projektverzeichnis
sys.path.insert(0, str(ROOT))

print("Working dir:", Path().resolve())
print("ROOT:", ROOT)

Working dir: C:\Users\aravi\PycharmProjects\aero-surrogate\notebooks
ROOT: C:\Users\aravi\PycharmProjects\aero-surrogate


In [12]:
from pathlib import Path
import sys

ROOT = Path(r"C:\Users\aravi\PycharmProjects\aero-surrogate")
sys.path.insert(0, str(ROOT))

from src.io_xfoil import parse_xfoil_polar_txt

polar_path = ROOT / "data" / "raw" / "naca2412_Re1e6_M0p2_polar.txt"
out_csv = ROOT / "data" / "processed" / "xfoil_polars.csv"

df = parse_xfoil_polar_txt(polar_path)
df.to_csv(out_csv, index=False)

print(df.head())
print("Rows:", len(df))
print("Saved to:", out_csv)



   alpha_deg      CL       CD      CM         Re  Mach    airfoil
0       -5.0 -0.3139  0.00862 -0.0582  1000000.0   0.2  NACA 2412
1       -4.0 -0.2024  0.00786 -0.0569  1000000.0   0.2  NACA 2412
2       -3.0 -0.0903  0.00721 -0.0559  1000000.0   0.2  NACA 2412
3       -2.0  0.0220  0.00671 -0.0551  1000000.0   0.2  NACA 2412
4       -1.0  0.1331  0.00623 -0.0543  1000000.0   0.2  NACA 2412
Rows: 20
Saved to: C:\Users\aravi\PycharmProjects\aero-surrogate\data\processed\xfoil_polars.csv


C:\Users\aravi\PycharmProjects\aero-surrogate\src\io_xfoil.py:52: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO("\n".join(lines[header_idx:data_start - 1] + lines[data_start:])),


In [13]:
from pathlib import Path
import sys
import pandas as pd

ROOT = Path.cwd().resolve().parents[0]     # notebook run from notebooks/
sys.path.insert(0, str(ROOT))

from src.io_xfoil import parse_xfoil_polar_txt

p1 = ROOT / "data" / "raw" / "naca2412_Re1e6_M0p2_polar.txt"
p2 = ROOT / "data" / "raw" / "naca2412_Re2e6_M0p2_polar.txt"

df1 = parse_xfoil_polar_txt(p1)
df2 = parse_xfoil_polar_txt(p2)

df = pd.concat([df1, df2], ignore_index=True)

out_csv = ROOT / "data" / "processed" / "xfoil_polars.csv"
df.to_csv(out_csv, index=False)

print("Saved:", out_csv)
print("Rows:", len(df))
print(df.groupby(["Re","Mach"]).size())
df.head()


Saved: C:\Users\aravi\PycharmProjects\aero-surrogate\data\processed\xfoil_polars.csv
Rows: 40
Re         Mach
1000000.0  0.2     20
2000000.0  0.2     20
dtype: int64


C:\Users\aravi\PycharmProjects\aero-surrogate\src\io_xfoil.py:52: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO("\n".join(lines[header_idx:data_start - 1] + lines[data_start:])),
C:\Users\aravi\PycharmProjects\aero-surrogate\src\io_xfoil.py:52: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO("\n".join(lines[header_idx:data_start - 1] + lines[data_start:])),


,alpha_deg,CL,CD,CM,Re,Mach,airfoil
0,-5.0,-0.3139,0.00862,-0.0582,1000000.0,0.2,NACA 2412
1,-4.0,-0.2024,0.00786,-0.0569,1000000.0,0.2,NACA 2412
2,-3.0,-0.0903,0.00721,-0.0559,1000000.0,0.2,NACA 2412
3,-2.0,0.0220,0.00671,-0.0551,1000000.0,0.2,NACA 2412
4,-1.0,0.1331,0.00623,-0.0543,1000000.0,0.2,NACA 2412


In [14]:
p2 = ROOT / "data" / "raw" / "naca2412_Re2e6_M0p2_polar.txt"


In [15]:
df = pd.read_csv(ROOT / "data" / "processed" / "xfoil_polars.csv")
print(df[["Re","Mach"]].drop_duplicates())
print("Rows:", len(df))


           Re  Mach
0   1000000.0   0.2
20  2000000.0   0.2
Rows: 40
